In [1]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os 
import spotipy
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics import pairwise_distances_argmin_min

client_id = os.environ.get('ce000ba19f9a4785a2835660a4618fe4')
client_secret = os.environ.get('81d154eca1e74751904777d465e90588')

client_credentials_manager = SpotifyClientCredentials('ce000ba19f9a4785a2835660a4618fe4', '81d154eca1e74751904777d465e90588')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



In [5]:
results = sp.search(q='artist:Ed Sheeran', type='track')
#if len(results['tracks']['items']) > 0:
    #track_id = results['tracks']['items']['id']
    #results = sp.recommendations(seed_tracks=[track_id], limit=20)
    #for track in results['tracks']:
        #print(track['name'], '-', track['artists'][0]['name'])
# extract the track ids
track_ids = [track['id'] for track in results['tracks']['items']]

# extract the audio features
audio_features = sp.audio_features(track_ids)

# store audio features in a dataframe
df = pd.DataFrame(audio_features)
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.788,0.859,2,-2.724,1,0.0856,0.28100,0.000000,0.0424,0.822,141.020,audio_features,50nfwKoDiSYg8zOCREWAm5,spotify:track:50nfwKoDiSYg8zOCREWAm5,https://api.spotify.com/v1/tracks/50nfwKoDiSYg...,https://api.spotify.com/v1/audio-analysis/50nf...,207853,4
1,0.599,0.448,8,-6.312,1,0.0232,0.16300,0.000000,0.1060,0.168,95.050,audio_features,0tgVpDi06FyKpA1z0VMD4v,spotify:track:0tgVpDi06FyKpA1z0VMD4v,https://api.spotify.com/v1/tracks/0tgVpDi06FyK...,https://api.spotify.com/v1/audio-analysis/0tgV...,263400,3
2,0.807,0.893,11,-3.745,0,0.0347,0.04510,0.000028,0.3660,0.537,126.011,audio_features,3rmo8F54jFF8OgYsqTxm5d,spotify:track:3rmo8F54jFF8OgYsqTxm5d,https://api.spotify.com/v1/tracks/3rmo8F54jFF8...,https://api.spotify.com/v1/audio-analysis/3rmo...,230747,4
3,0.807,0.893,11,-3.745,0,0.0347,0.04510,0.000028,0.3660,0.537,126.011,audio_features,3PEhQpIn1ByReSBi4Lu06e,spotify:track:3PEhQpIn1ByReSBi4Lu06e,https://api.spotify.com/v1/tracks/3PEhQpIn1ByR...,https://api.spotify.com/v1/audio-analysis/3PEh...,230747,4
4,0.522,0.585,7,-7.205,1,0.1290,0.00208,0.000000,0.0909,0.411,97.990,audio_features,6ux363TEy1w5nYecMC3bnm,spotify:track:6ux363TEy1w5nYecMC3bnm,https://api.spotify.com/v1/tracks/6ux363TEy1w5...,https://api.spotify.com/v1/audio-analysis/6ux3...,191896,4
5,0.693,0.643,4,-5.867,0,0.0828,0.13800,0.000000,0.1080,0.477,95.055,audio_features,2qteiR2DXBlkhU0jBqMCMJ,spotify:track:2qteiR2DXBlkhU0jBqMCMJ,https://api.spotify.com/v1/tracks/2qteiR2DXBlk...,https://api.spotify.com/v1/audio-analysis/2qte...,163450,4
6,0.522,0.585,7,-7.205,1,0.1290,0.00208,0.000000,0.0909,0.411,97.990,audio_features,5MjYtFVYIQwliXrRlVxEUy,spotify:track:5MjYtFVYIQwliXrRlVxEUy,https://api.spotify.com/v1/tracks/5MjYtFVYIQwl...,https://api.spotify.com/v1/audio-analysis/5MjY...,191896,4
7,0.598,0.448,8,-6.312,1,0.0232,0.16200,0.000000,0.1060,0.168,95.009,audio_features,5aBKzKFNX5wL1JMnG5V1lw,spotify:track:5aBKzKFNX5wL1JMnG5V1lw,https://api.spotify.com/v1/tracks/5aBKzKFNX5wL...,https://api.spotify.com/v1/audio-analysis/5aBK...,263400,3
8,0.788,0.859,2,-2.724,1,0.0856,0.28100,0.000000,0.0424,0.822,141.020,audio_features,5UbC4eA9xIgmnaGHLb3aSv,spotify:track:5UbC4eA9xIgmnaGHLb3aSv,https://api.spotify.com/v1/tracks/5UbC4eA9xIgm...,https://api.spotify.com/v1/audio-analysis/5UbC...,207853,4
9,0.567,0.211,0,-9.975,1,0.0315,0.95200,0.000052,0.2750,0.289,134.019,audio_features,4oorALETwyuLgOx3hqqjUb,spotify:track:4oorALETwyuLgOx3hqqjUb,https://api.spotify.com/v1/tracks/4oorALETwyuL...,https://api.spotify.com/v1/audio-analysis/4oor...,185587,4


In [6]:
# create a function to automate work
def get_audio_features(artist):
    # get tracks from artist
    results = sp.search(q=f'artist:{artist}', limit=50)
    # extract the track ids
    track_ids = [track['id'] for track in results['tracks']['items']]
    song_names = [track['name'] for track in results['tracks']['items']]
    # extract the audio features
    audio_features = sp.audio_features(track_ids)
    # store audio features in a dataframe
    df = pd.DataFrame(audio_features)
    df['artist'] = artist
    df['song name'] = song_names
    return df

In [7]:
# create loop and select artists you want to get songs from

artists = ['Dua Lipa', 'Ava Max', 'Billie Eilish']

df = pd.DataFrame()

for artist in artists:
    df_artist = get_audio_features(artist)
    df = pd.concat([df, df_artist])
    
df = df.reset_index(drop=True)
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist,song name
0,0.702,0.825,6,-3.787,0,0.0601,0.00883,0.00000,0.0674,0.915,102.977,audio_features,5nujrmhLynf4yMoMtj8AQF,spotify:track:5nujrmhLynf4yMoMtj8AQF,https://api.spotify.com/v1/tracks/5nujrmhLynf4...,https://api.spotify.com/v1/audio-analysis/5nuj...,203064,4,Dua Lipa,Levitating (feat. DaBaby)
1,0.695,0.884,6,-2.278,0,0.0753,0.05610,0.00000,0.2130,0.914,103.014,audio_features,39LLxExYz6ewLAcYrzQQyP,spotify:track:39LLxExYz6ewLAcYrzQQyP,https://api.spotify.com/v1/tracks/39LLxExYz6ew...,https://api.spotify.com/v1/audio-analysis/39LL...,203808,4,Dua Lipa,Levitating
2,0.793,0.793,11,-4.521,0,0.0830,0.01230,0.00000,0.0951,0.679,123.950,audio_features,3PfIrDoz19wz7qK7tYeu62,spotify:track:3PfIrDoz19wz7qK7tYeu62,https://api.spotify.com/v1/tracks/3PfIrDoz19wz...,https://api.spotify.com/v1/audio-analysis/3PfI...,183290,4,Dua Lipa,Don't Start Now
3,0.431,0.224,6,-8.683,1,0.0379,0.94800,0.00000,0.1120,0.401,87.122,audio_features,7lkWt8qbnJpUb9XTIfzN9O,spotify:track:7lkWt8qbnJpUb9XTIfzN9O,https://api.spotify.com/v1/tracks/7lkWt8qbnJpU...,https://api.spotify.com/v1/audio-analysis/7lkW...,230480,5,Dua Lipa,Homesick
4,0.793,0.793,11,-4.521,0,0.0830,0.01230,0.00000,0.0951,0.679,123.950,audio_features,6IXoRzbV65nIc4sbPd2o9R,spotify:track:6IXoRzbV65nIc4sbPd2o9R,https://api.spotify.com/v1/tracks/6IXoRzbV65nI...,https://api.spotify.com/v1/audio-analysis/6IXo...,183290,4,Dua Lipa,Don't Start Now
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.332,0.225,0,-8.697,1,0.0348,0.76700,0.00349,0.1280,0.297,81.055,audio_features,4lWe7kBpYunzkSBDstI05i,spotify:track:4lWe7kBpYunzkSBDstI05i,https://api.spotify.com/v1/tracks/4lWe7kBpYunz...,https://api.spotify.com/v1/audio-analysis/4lWe...,298899,3,Billie Eilish,Happier Than Ever
146,0.704,0.225,6,-14.454,0,0.0994,0.90100,0.65700,0.1060,0.243,120.006,audio_features,19zUqs9QYP9h1esZVBZEbN,spotify:track:19zUqs9QYP9h1esZVBZEbN,https://api.spotify.com/v1/tracks/19zUqs9QYP9h...,https://api.spotify.com/v1/audio-analysis/19zU...,245426,4,Billie Eilish,everything i wanted
147,0.671,0.336,10,-8.494,0,0.2410,0.70500,0.00751,0.0577,0.518,75.051,audio_features,4OmXHiyeFO57DWFSmnXozy,spotify:track:4OmXHiyeFO57DWFSmnXozy,https://api.spotify.com/v1/tracks/4OmXHiyeFO57...,https://api.spotify.com/v1/audio-analysis/4OmX...,212496,4,Billie Eilish,Lost Cause
148,0.485,0.411,7,-8.461,1,0.0397,0.73500,0.00000,0.1160,0.247,170.300,audio_features,6RHi06pPWJNpizAnENkjn2,spotify:track:6RHi06pPWJNpizAnENkjn2,https://api.spotify.com/v1/tracks/6RHi06pPWJNp...,https://api.spotify.com/v1/audio-analysis/6RHi...,203569,3,Billie Eilish,idontwannabeyouanymore


In [8]:
numerical = df.select_dtypes([np.number])
numerical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      150 non-null    float64
 1   energy            150 non-null    float64
 2   key               150 non-null    int64  
 3   loudness          150 non-null    float64
 4   mode              150 non-null    int64  
 5   speechiness       150 non-null    float64
 6   acousticness      150 non-null    float64
 7   instrumentalness  150 non-null    float64
 8   liveness          150 non-null    float64
 9   valence           150 non-null    float64
 10  tempo             150 non-null    float64
 11  duration_ms       150 non-null    int64  
 12  time_signature    150 non-null    int64  
dtypes: float64(9), int64(4)
memory usage: 15.4 KB


In [9]:
scaler = StandardScaler()

num_prep = scaler.fit_transform(numerical)

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(num_prep)

clusters = kmeans.predict(num_prep)

scaled_df = pd.DataFrame(num_prep, columns=numerical.columns)
scaled_df['song name'] = df['song name']
scaled_df['artist'] = df['artist']
scaled_df['cluster'] = clusters
scaled_df

C:\Users\thuyl\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song name,artist,cluster
0,0.567497,0.762264,0.257423,0.679308,-0.675508,-0.340578,-0.784215,-0.285590,-0.822369,1.845483,-0.769628,0.098531,0.103835,Levitating (feat. DaBaby),Dua Lipa,0
1,0.516270,0.991308,0.257423,1.041947,-0.675508,-0.142005,-0.650645,-0.285590,0.754611,1.841503,-0.767934,0.120701,0.103835,Levitating,Dua Lipa,0
2,1.233447,0.638036,1.687552,0.502915,-0.675508,-0.041413,-0.774409,-0.285590,-0.522353,0.906273,0.190866,-0.490698,0.103835,Don't Start Now,Dua Lipa,0
3,-1.415717,-1.570879,0.257423,-0.497288,1.480368,-0.630598,1.869567,-0.285590,-0.339310,-0.200083,-1.495735,0.915478,3.997641,Homesick,Dua Lipa,2
4,1.233447,0.638036,1.687552,0.502915,-0.675508,-0.041413,-0.774409,-0.285590,-0.522353,0.906273,0.190866,-0.490698,0.103835,Don't Start Now,Dua Lipa,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,-2.140212,-1.566997,-1.458731,-0.500652,1.480368,-0.671096,1.358121,-0.254607,-0.166016,-0.613972,-1.773584,2.954239,-3.789971,Happier Than Ever,Billie Eilish,2
146,0.582133,-1.566997,0.257423,-1.884162,-0.675508,0.172836,1.736760,5.547129,-0.404296,-0.828876,0.010244,1.360841,0.103835,everything i wanted,Billie Eilish,1
147,0.340635,-1.136084,1.401526,-0.451868,-0.675508,2.022695,1.182930,-0.218918,-0.927428,0.265542,-2.048548,0.379588,0.103835,Lost Cause,Billie Eilish,1
148,-1.020538,-0.844926,0.543449,-0.443937,1.480368,-0.607083,1.267700,-0.285590,-0.295987,-0.812957,2.313544,0.113579,-3.789971,idontwannabeyouanymore,Billie Eilish,2


In [12]:

scaled_df.groupby(['cluster', 'artist'], as_index=False).count().sort_values(['cluster', 'key'], ascending=[True, False])[['artist', 'cluster', 'key']].reset_index(drop=True)

,artist,cluster,key
0,Ava Max,0,50
1,Dua Lipa,0,47
2,Billie Eilish,0,3
3,Billie Eilish,1,17
4,Billie Eilish,2,30
5,Dua Lipa,2,3


In [11]:
song_name = input('Choose a song: ')
results = sp.search(q=f'track:{song_name}', limit=1)
track_id = results['tracks']['items'][0]['id']
audio_features = sp.audio_features(track_id)

df_ = pd.DataFrame(audio_features)
new_features = df_[numerical.columns]

scaled_num = scaler.transform(new_features)
cluster = kmeans.predict(scaled_num)

filtered_df = np.array(scaled_df[scaled_df['cluster'] == cluster[0]][numerical.columns], order="C")
closest, _ = pairwise_distances_argmin_min(scaled_num, filtered_df)

Choose a song: flowers


In [13]:
scaled_df.loc[closest[0]]['song name'], scaled_df.loc[closest[0]]['artist']

('Fever', 'Dua Lipa')

In [16]:

def recommend_song():
    # get song id
    song_name = input('Choose a song: ')
    results = sp.search(q=f'track:{song_name}', limit=1)
    track_id = results['tracks']['items'][0]['id']
    # get song features with the obtained id
    audio_features = sp.audio_features(track_id)
    # create dataframe
    df_ = pd.DataFrame(audio_features)
    new_features = df_[numerical.columns]
    # scale features
    scaled_num = scaler.transform(new_features)
    # predict cluster
    cluster = kmeans.predict(scaled_num)
    # filter dataset to predicted cluster
    filtered_df = np.array(scaled_df[scaled_df['cluster'] == cluster[0]][numerical.columns], order="C")
    # get closest song from filtered dataset
    closest, _ = pairwise_distances_argmin_min(scaled_num, filtered_df)
    # return it in a readable way
    print('\n[RECOMMENDED SONG]')
    return ' - '.join([scaled_df.loc[closest]['song name'].values[0], scaled_df.loc[closest]['artist'].values[0]])

In [17]:
recommend_song()


Choose a song: love yourself

[RECOMMENDED SONG]


'Levitating - Dua Lipa'